<a href="https://colab.research.google.com/github/ducthuy-ng/computer_vision_sem_241/blob/main/DLACV_Ass1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.set_default_device(device)

## Custom Model for CIFAR-10 classification

1. **(3pts)** Create a Resnet-18 or RegNet-Y or MobileNet for CIFAR-10 classification

 * (a) Train model from scratch without using pretrained weights from ImageNet; evaluate.

 * (b) Train model using pretrained weights from ImageNet (fine-tune transfer learning); evaluate. The performance must higher than results from (a). Provide discussion for the results.

2. **(4pts)** Repeat requirements from (1-a) and (1-b) but add data augmentation with methods from https://github.com/albumentations-team/albumentations . The performance must higher than results from (1) Provide discussion for the results.

3. **(3pts)** Find best learning rate for models from (1) and (2) with learning rate finder from https://github.com/surmenok/keras_lr_finder or other similar approaches. Provide discussion for the results.


In [ ]:
### For PyTorch-based coding
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
trainloader = DataLoader(
    trainset, batch_size=batch_size, shuffle=True, num_workers=2, generator=torch.Generator(device=device)
)

testset = torchvision.datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)
testloader = DataLoader(
    testset, batch_size=batch_size, shuffle=False, num_workers=2, generator=torch.Generator(device=device)
)
# Exercise 1
## Pre-trained ResNet-18
from torchvision.models import resnet18

resnet_18_model = resnet18(weights="IMAGENET1K_V1").to(device)
Transfer learning
# Freeze all pre-trained params
for param in resnet_18_model.parameters():
    param.requires_grad = False

# Alter output layer
resnet_18_model.fc = torch.nn.Linear(in_features=512, out_features=10, bias=True)
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam( # type: ignore
    params=resnet_18_model.fc.parameters(),
    lr=0.001,
)
EPOCH_NUMS = 100

for epoch in range(EPOCH_NUMS):
    resnet_18_model.train()

    # Initialize the running loss and accuracy
    running_loss = 0.0
    running_corrects = 0

    for inputs, labels in trainloader:
        optimizer.zero_grad()
        outputs = resnet_18_model(inputs.to(device))
        _, predictions = torch.max(outputs, 1)
        loss = loss_function(outputs, labels)

        loss.backward()
        optimizer.step()

        # Update the running loss and accuracy
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(predictions == labels.data)

    # Calculate the train loss and accuracy
    train_loss = running_loss / len(trainloader)
    train_acc = running_corrects / len(trainloader)

    # Set the model to evaluation mode
    resnet_18_model.eval()

    # Initialize the running loss and accuracy
    running_loss = 0.0
    running_corrects = 0

    # Iterate over the batches of the validation loader
    with torch.no_grad():
        for inputs, labels in testloader:
            # Forward pass
            outputs = resnet_18_model(inputs)
            _, predictions = torch.max(outputs, 1)
            loss = loss_function(outputs, labels)

            # Update the running loss and accuracy
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(predictions == labels.data)

    # Calculate the validation loss and accuracy
    test_loss = running_loss / len(testloader)
    test_acc = running_corrects / len(testloader)

    print(
        f"Epoch [{epoch + 1}/{EPOCH_NUMS}], train loss: {train_loss:.4f}, train acc: {train_acc:.4f}, test loss: {test_loss:.4f}, test acc: {test_acc:.4f}"
    )
